<a href="https://colab.research.google.com/github/yashgupta417/imdb/blob/master/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [0]:
vocab_size=5000
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=vocab_size)

In [7]:
#data is already preprocessed
X_train[1][:10]#first 10 words of review 1

[1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463]

In [9]:
max_review_length=600
X_train=sequence.pad_sequences(X_train,maxlen=max_review_length)
X_test=sequence.pad_sequences(X_test,maxlen=max_review_length)
X_train.shape

(25000, 600)